In [6]:
import timeit

start_time = timeit.default_timer()
x = 0
while x < 10_000_000:
    x += 1

elapsed_time = timeit.default_timer() - start_time
print(f"Time needed: {elapsed_time:.6f} seconds")

Time needed: 0.492637 seconds


In [4]:
0.88 / 0.0015

586.6666666666666

In [3]:
# convert array of edges to adjecency list
from collections import defaultdict

D = defaultdict(list)

for u, v in A:
    D[u].append(v)

D


defaultdict(list, {0: [1, 3], 1: [2], 3: [4, 6, 7], 4: [2, 5], 5: [2]})

In [4]:
import networkx as nx

G = nx.Graph()
G.add_node('A2')
G.add_nodes_from(['A3','A4','A5'])

G.add_edge('A2', 'A3')
G.add_edges_from([('A3','A4'),('A4','A5')])

for i in range(2, 6):
    G.add_edge(f'B{i}', f'C{i}')
    if 2<i<5:
        G.add_edge(f'B{i}', f'B{i+1}')
    if i<5:
        G.add_edge(f'C{i}', f'C{i+1}')

In [5]:
print(G.number_of_nodes(), 'nodes')
print(G.number_of_edges(), 'edges')
print(list(G['C3']))
print(list(G.edges('C3')))

12 nodes
12 edges
['C2', 'B3', 'C4']
[('C3', 'C2'), ('C3', 'B3'), ('C3', 'C4')]


In [6]:
from collections import deque
import heapq

class Node:
    def __init__(self, val, next=None) -> None:
        self.next = next
        self.value = val
        
class Stack:
    def __init__(self) -> None:
        self.top = None
    
    def is_empty(self):
        return self.top is None
    
    def push(self, val):
        self.top = Node(val=val, next=self.top)
        
    def pop(self):
        if self.is_empty():
            raise RuntimeError('Stack is empty!')
        
        val = self.top.value
        self.top = self.top.next
        return val
    
def dfs_search(G, src):
    seen = set()
    node_from = {}
    
    stack = Stack()
    stack.push(src)
    
    while not stack.is_empty():
        curr_node = stack.pop()
        
        for node in list(G[curr_node]):
            if node not in seen:
                node_from[node] = curr_node
                stack.push(node)
                seen.add(node)
    
    return node_from

def bff_search(G, src):
    seen = set()
    node_from = {}
    
    queue = deque()
    queue.appendleft(src)
    
    while not len(queue) <= 0:
        curr_node = queue.pop()
        
        for node in list(G[curr_node]):
            if node not in seen:
                node_from[node] = curr_node
                queue.appendleft(node)
                seen.add(node)
                
    return node_from

def path_to(node_from, src, target):
    if not target in node_from:
        raise ValueError('Unreachable')
    
    path = []
    curr_node = target
    
    while curr_node!=src:
        path.append(curr_node)
        curr_node=node_from[curr_node]
    
    path.append(src)
    path.reverse()
    return path

In [7]:
import random

G = nx.Graph()

G.add_nodes_from(range(20))

for _ in range(30):
    node1 = random.randint(0, 19)
    node2 = random.randint(0, 19)
    if node1 != node2:
        G.add_edge(node1, node2)

In [8]:
node_from = bff_search(G, 10)
print(node_from)
path = path_to(node_from, 10, 12)
print(path)

{11: 10, 2: 10, 10: 11, 14: 2, 16: 2, 6: 2, 4: 14, 0: 16, 17: 4, 5: 4, 18: 4, 12: 0, 19: 17, 15: 5, 1: 5, 7: 18, 13: 12, 8: 19, 3: 7}
[10, 2, 16, 0, 12]


In [9]:
import networkx as nx


G = nx.DiGraph()

G.add_edges_from([(1,2), (1,3), (3,4), (3,5), (5,6), (7,8)])


    
def dfs_search(G, src):
    seen = set()
    node_from = {}
    
    stack = Stack()
    stack.push(src)
    
    while not stack.is_empty():
        curr_node = stack.pop()
        
        for node in list(G[curr_node]):
            if node not in seen:
                node_from[node] = curr_node
                stack.push(node)
                seen.add(node)
    
    return node_from

In [10]:
list(nx.neighbors(G, 2))

[]

In [11]:
g_cycle = nx.DiGraph()
g_cycle.add_edges_from([(1,2), (2,3), (1,3), (4,2), (3,4)])


def has_cycle_2squared(graph):
    
    for node in g_cycle:
        this_run_seen = set()
        stack = Stack()
        
        stack.push(node)
        
        while not stack.is_empty():
            curr_node = stack.pop()
            
            for n in nx.neighbors(graph, curr_node):
                if n == node:
                    return True
                elif n not in this_run_seen:
                    this_run_seen.add(n)
                    stack.push(n)
    return False


def has_cycle(graph):
    def dfs(node, temp_seen, perm_seen):
        if node in perm_seen:
            return False
        if node in temp_seen:
            return True
        
        temp_seen.add(node)
        for successor in graph.successors(node):
            if dfs(successor, temp_seen, perm_seen):
                return True
        
        temp_seen.remove(node)
        perm_seen.add(node)
        return False
    
    temp_seen = set()
    perm_seen = set() 
    for node in graph.nodes():
        if node not in perm_seen:
            if dfs(node, temp_seen, perm_seen):
                return True
    
    return False
                
has_cycle(g_cycle)

True

In [12]:
g_cycle = nx.DiGraph()
g_cycle.add_edges_from([(1,2), (2,3), (1,3), (4,2), (4,3)])

def topological_sort(graph):
    seen = set()
    postorder = []
    
    def dfs(node):
        seen.add(node)
        for successor in graph.successors(node):
            if successor not in seen:
                dfs(successor)
        postorder.append(node)
    
    for node in graph.nodes():
        if node not in seen:
            dfs(node)
    
    return reversed(postorder)

list(topological_sort(g_cycle))

[4, 1, 2, 3]

In [13]:
import random
import networkx as nx

G = nx.DiGraph()

# Add 20 nodes
G.add_nodes_from(range(1, 21))

# Randomly add edges with weights
for _ in range(40):  # Add 40 random edges
    u = random.randint(1, 20)
    v = random.randint(1, 20)
    if u != v:  # Avoid self-loops
        weight = round(random.uniform(1.0, 10.0), 2)  # Random weight between 1.0 and 10.0
        G.add_edge(u, v, weight=weight)

In [14]:
dfs_search(G, 2)

{3: 2,
 15: 2,
 17: 2,
 18: 2,
 10: 17,
 1: 17,
 14: 1,
 5: 10,
 11: 10,
 4: 11,
 19: 4,
 12: 19,
 16: 3,
 20: 3,
 2: 20}

In [15]:
import heapq

pq = []
heapq.heappush(pq, (50, 'Z'))
heapq.heappush(pq, (20, 'B'))
heapq.heappush(pq, (10, 'A'))

heapq.heappop(pq)

(10, 'A')

In [16]:
import heapq

class PQUpdate:
    def __init__(self) -> None:
        self.heap = []
        self.newest_values = {}
        
    def is_empty(self):
        return len(self.heap) <= 0
        
    def push(self, priority, value):
        '''
        Pushing a value already present will update newest_values, so the old couple (priority, value) won't be popped.
        '''
        self.newest_values[value] = priority
        heapq.heappush(self.heap, (priority, value))
    
    def pop(self):
        ''' 
        Pop and return the value with the lest priority. If the (priority, value) is not present in newest_values, it'll continue to pop.
        '''
        while not self.is_empty():
            priority, value = heapq.heappop(self.heap)
            
            if priority == self.newest_values[value]:
                #del self.newest_values[value]
                return (priority, value)
            

def reconstruct_path(predecessors, target):
    """
    Reconstruct the shortest path from the target node back to the source.
    """
    path = []
    while target in predecessors:
        path.append(target)
        target = predecessors[target] # Go to the predecessor
    return path[::-1] # Reverse to get the path from source to target


def dijkstra(graph, src, target):
    
    distances = {src: 0}
    predecessors = {}
    heap = PQUpdate()
    heap.push(0, src)
    
    while not heap.is_empty():
        prev_distance, node = heap.pop()
        
        if node == target:
            path = reconstruct_path(predecessors, target)
            return [src] + path, distances[target]
        
        for successor in graph.successors(node):
            curr_distance = graph[node][successor]['weight']
            tot_distance = prev_distance + curr_distance
            
            if successor not in distances:
                # keeps track of the distance and the parent node
                distances[successor] = tot_distance
                predecessors[successor] = node
                # push the successor into the heap
                heap.push(tot_distance, successor)
            
            
            elif distances[successor] > tot_distance:
                
                distances[successor] = tot_distance
                predecessors[successor] = node
                heap.push(tot_distance, successor)
    
    return None

In [17]:
dijkstra(G, 2, 11)

([2, 17, 10, 11], 14.899999999999999)

In [18]:
def dfs_recoursive(graph, src):
    visited = set()
    node_from = {}
    
    def dfs_r(node):
        if node in visited:
            return
        
        visited.add(node)
        for successor in graph.successors(node):
            dfs_r(successor)
            node_from[successor] = node
    
    dfs_r(src)
    
    return node_from

dfs_recoursive(G, 2)

{18: 2,
 10: 17,
 17: 2,
 1: 20,
 14: 1,
 5: 10,
 15: 2,
 12: 20,
 11: 20,
 19: 4,
 4: 11,
 16: 3,
 2: 20,
 20: 3,
 3: 2}

In [19]:
def gen_nodes(node_from, src, target):
    if not target in node_from:
        raise ValueError('Unreachable')
    
    def r_gen_nodes(node):
        if node == target:
            yield node
            return
        
        yield node
        yield from r_gen_nodes(node_from[node])
    
    yield from r_gen_nodes(src)

node_from = dfs_recoursive(G, 2)
for node in gen_nodes(node_from, 17, 2):
    print(node)

17
2


In [20]:
g_cycle = nx.DiGraph()
g_cycle.add_edges_from([(1,2), (2,3), (1,3), (4,2), (3,4)])


def has_cycle(graph):
    
    first_node_of_cycle = None
    
    def dfs(node, temp_seen, perm_seen):
        nonlocal first_node_of_cycle
        if node in perm_seen:
            return False
        if node in temp_seen:
            cycle.append(node)
            first_node_of_cycle = node
            return True
        
        temp_seen.add(node)
        for successor in graph.successors(node):
            if dfs(successor, temp_seen, perm_seen):
                
                if first_node_of_cycle and first_node_of_cycle != node:
                    cycle.append(node)
                elif first_node_of_cycle and first_node_of_cycle == node:
                    first_node_of_cycle = None
                return True
        
        temp_seen.remove(node)
        perm_seen.add(node)
        return False
    
    temp_seen = set()
    perm_seen = set()
    cycle = []
    for node in graph.nodes():
        if node not in perm_seen:
            if dfs(node, temp_seen, perm_seen):
                return cycle
    
    return False
                
has_cycle(g_cycle)

[2, 4, 3]

In [1]:
import geopandas as gpd

shapefile_path = r"EOTMAJROADS_ARC\EOTMAJROADS_ARC.shp"
roads = gpd.read_file(shapefile_path)

In [2]:
roads.tail(1)['geometry']

22099    LINESTRING (229767.768 901426.514, 229769.59 9...
Name: geometry, dtype: geometry

In [39]:
G = nx.DiGraph()

# Add edges to the graph
for _, row in roads.iterrows():
    geometry = row.geometry
    
    # Ensure the geometry is valid and is a LineString
    if geometry and geometry.geom_type == "LineString":
        coords = list(geometry.coords)
        
        # Add edges between consecutive points in the LineString
        for i in range(len(coords) - 1):
            start = coords[i]
            end = coords[i + 1]
            
            # Use SHAPE_LEN or calculate distance as weight
            G.add_edge(
                start,
                end,
                length=row.SHAPE_LEN,
                street_name=row.STREET_NAM,  # Add street name as an attribute
                attributes=row.to_dict()    # Store additional attributes
            )

In [41]:
i = 0
for node in G.nodes():
    i +=1
    
    if i > 5:
        break
    
    print(node)

(225071.875, 900529.4373999983)
(225042.1093999967, 900560.3126000017)
(225035.41009999812, 900567.4816999994)
(224999.9998999983, 900605.3751999997)
(224971.375, 900634.125)


In [42]:
source = (225071.875, 900529.4373999983)
target = (224971.375, 900634.125)

nx.shortest_path(G, source, target, weight='length')

[(225071.875, 900529.4373999983),
 (225042.1093999967, 900560.3126000017),
 (225035.41009999812, 900567.4816999994),
 (224999.9998999983, 900605.3751999997),
 (224971.375, 900634.125)]